# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

In [5]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [6]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [7]:
standup_json='standup_2.json'

In [8]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [9]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 2. Chemicals to Gene

What chemicals are associated with SMS (UniProtKB:P52788)

Github Issue: https://github.com/NCATSTranslator/testing/issues/10


The query as run through the ARS:

In [10]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "UniProtKB:P52788",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:ChemicalSubstance"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [11]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [12]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-02 00:00:00

Done ara-aragorn 63
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=d187edcd-f4a1-42e8-9248-eee0cb161fdb


### 2021-03-16 00:00:00

Done ara-aragorn 119
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=5ae7b3a8-ab4d-4282-a027-80a7116f1265


### 2021-03-19 00:00:00

Done ara-aragorn 128
Done ara-aragorn-exp 86
https://arax.ncats.io/?source=ARS&id=0f16dbfa-444a-409f-a7b2-9d11506b0d1c


### 2021-03-25 00:00:00

Done ara-aragorn 128
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=8675552b-34b1-46eb-a4f0-3e34179a1e12


## Strider Direct

In [13]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 37 results in 0:01:30.763602.


In [14]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e01,https://translator.broadinstitute.org/molepro/...,37


In [15]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

I think we should be getting results from more than one source for this query.  It's also slow.   90 seconds for a one-hop query that produces 40 results.

## ARAGORN 

In [16]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")

KeyError: 'results'

In [17]:
aragorn_result

{'message': {'query_graph': {'nodes': {'n0': {'id': 'UniProtKB:P52788',
     'category': 'biolink:Gene',
     'is_set': False},
    'n1': {'category': 'biolink:ChemicalSubstance', 'is_set': False}},
   'edges': {'e01': {'subject': 'n0', 'object': 'n1'}}}},
 'error': 'Error detected: Got an empty answer from Answer coalesce, aborting.'}

In [ ]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

### ARAGORN Assessment

ARAGORN is failing in AC, for no clear reason.  There may not be enrichment of these chemicals in graph.  But I bet there is in property.   And I bet there is in graph for that matter.